In [6]:
from src.XMLUtils import make_xml_file, get_sections
from src.OpenAIUtils import get_message

dir_path = "/home/paper_translator/data/Learning_Transferable_Visual_Models_From_Natural_Language_Supervision"
pdf_name = "Learning_Transferable_Visual_Models_From_Natural_Language_Supervision"

In [7]:
root = make_xml_file(dir_path=dir_path, pdf_name=pdf_name, is_debug=True)

In [8]:
sections = get_sections(root=root)

In [9]:
section = sections[0]

In [10]:
SYSTEM = """
### 指示 ###
論文の内容を理解した上で，重要なポイントを箇条書きで3点書いてください。

### 箇条書きの制約 ###
- 最大3個
- 日本語
- 箇条書き1個を100文字以内

### 対象とする論文の内容 ###
{text}

### 出力形式 ###
- 箇条書き1
- 箇条書き2
- 箇条書き3
"""

response = get_message(text=section.body, system=SYSTEM)

In [11]:
print(response)

- Pre-training methods that learn from raw text have revolutionized NLP and web-scale collections of text provide more aggregate supervision than high-quality crowd-labeled NLP datasets. 
- Natural language supervision can be used in computer vision for narrow and well-targeted tasks, such as predicting ImageNet-related hashtags on Instagram images and pre-training models to predict the classes of the noisily labeled JFT-300M dataset. 
- CLIP, a method of learning from natural language supervision, is an efficient way to perform a wide set of tasks during pre-training, including OCR, geo-localization, and action recognition. CLIP is more efficient at zero-shot transfer than an image caption baseline and is more robust than equivalent accuracy supervised ImageNet models.


# fugumt-en-ja を使って翻訳する

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
model_name = "staka/fugumt-en-ja"
#model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#summarizer = pipeline("summarization", model="kworts/BARTxiv")
translator = pipeline("translation", model=model_name, tokenizer=tokenizer)

In [15]:
translated_text = translator(response)[0]["translation_text"]

In [16]:
print(translated_text)

言語スーパーバイサーが使用可能な学習方法は,従来よりも優れたものである反省が,新たな方法では,より低い性能を示すことができる。.. MRIの学習に先立つような,新たな対人関係の可能な,新たな画像生成のアプローチ。この技術を用いるための,トレーニングに有効な,新たな手法を提案する,新たな手法を提案する,新たな,新たな,新たな,新たな,新たな手法を提案する,新たな,新たな,新たな,新たな,新たな,新たな手法を提案する,新たな,新たな,新たな,より効率的な,より効率的な,より効率的な,より効率的な,より優れた,優れた,優れた,優れた,最先端の,学習技術手法を提案することができる,という,その目的から,現在30年以上にわたって,より優れたデータから,より優れた,より効率的な,より効率的な,より効率的な,より効率的な,より優れた,より優れた,より強力な,より優れた,という特徴をも持つことができる,という特徴が,その上では,現在,そのようにしていまだ見受けられるようにも,そのように,そのようにしていまだにそのようにしていまだにそのようにしては,そのようにしては,そのようにして,そのようにしていまだにそのようにしていまだにそのようにしていまだにそのようにしては,そのようにしては,そのようにしていまだ,そのようにしていまだにそのようにしていまだに,そのようにしていまだに,そのようにしていちが,そのように,そのように,そのようにしていまだにそのようにしては,そのように,そのようにしていまだにそのようにしていちが,そのようにしてはいちが,そのように,そのように,そのように,そのように,そのように,そのように,そのように,そのように,そのように,そのように,4....で,そのように、4万万万万,というように,というように,そのように,そのように,そのようにという,4万,4万,4万万万万万,という,という,という,という,という,という,という,4という,という,という,という,という,という,という,という,という,という,という,という,という,という,という,という、という、という、という、という、という、という、という、という、という,という、という、という、という、という、という、という、という、という、という、という、という、という、という、という、という、という、という、とい

# Weblab-10Bを使って翻訳する

In [1]:
from auto_gptq import AutoGPTQForCausalLM
from transformers import AutoTokenizer, pipeline

In [2]:
# トークナイザーとモデルの準備
quantized_model_dir = "dahara1/weblab-10b-instruction-sft-GPTQ"
model_basename = "gptq_model-4bit-128g"

In [3]:
summarizer = pipeline("summarization", model="kworts/BARTxiv")
tokenizer = AutoTokenizer.from_pretrained(
    quantized_model_dir, use_fast=True
)

In [4]:
model = AutoGPTQForCausalLM.from_quantized(
    quantized_model_dir,
    model_basename=model_basename,
    use_safetensors=True,
    device="cuda:0",
)

GPTNeoXGPTQForCausalLM hasn't fused attention module yet, will skip inject fused attention.
GPTNeoXGPTQForCausalLM hasn't fused mlp module yet, will skip inject fused mlp.
GPTNeoXGPTQForCausalLM hasn't fused mlp module yet, will skip inject fused mlp.


In [19]:
prompt_text = response
print(prompt_text)

- Pre-training methods that learn from raw text have revolutionized NLP and web-scale collections of text provide more aggregate supervision than high-quality crowd-labeled NLP datasets. 
- Natural language supervision can be used in computer vision for narrow and well-targeted tasks, such as predicting ImageNet-related hashtags on Instagram images and pre-training models to predict the classes of the noisily labeled JFT-300M dataset. 
- CLIP, a method of learning from natural language supervision, is an efficient way to perform a wide set of tasks during pre-training, including OCR, geo-localization, and action recognition. CLIP is more efficient at zero-shot transfer than an image caption baseline and is more robust than equivalent accuracy supervised ImageNet models.


In [61]:
PROMPT_TEMPLATE = """
### 指示 ###
対象とする文章を制約に基づいて翻訳してください。

### 対象とする文章 ###
{prompt_text}

### 文章の制約 ###
- 日本語で出力してください。
- 箇条書きの文章ごとに翻訳してください。
- 1つの項目ごとに100文字以内で翻訳してください。
- 箇条書きで出力してください。

### 出力 ###
"""

In [62]:
print(PROMPT_TEMPLATE.format(prompt_text=prompt_text))


### 指示 ###
対象とする文章を制約に基づいて翻訳してください。

### 対象とする文章 ###
- Pre-training methods that learn from raw text have revolutionized NLP and web-scale collections of text provide more aggregate supervision than high-quality crowd-labeled NLP datasets. 
- Natural language supervision can be used in computer vision for narrow and well-targeted tasks, such as predicting ImageNet-related hashtags on Instagram images and pre-training models to predict the classes of the noisily labeled JFT-300M dataset. 
- CLIP, a method of learning from natural language supervision, is an efficient way to perform a wide set of tasks during pre-training, including OCR, geo-localization, and action recognition. CLIP is more efficient at zero-shot transfer than an image caption baseline and is more robust than equivalent accuracy supervised ImageNet models.

### 文章の制約 ###
- 日本語で出力してください。
- 箇条書きの文章ごとに翻訳してください。
- 1つの項目ごとに100文字以内で翻訳してください。
- 箇条書きで出力してください。

### 出力 ###



In [63]:
tokens = (
    tokenizer(PROMPT_TEMPLATE.format(prompt_text=prompt_text), return_tensors="pt")
    .to("cuda:0")
    .input_ids
)
print(tokens)

tensor([[  187,  4118,   209, 34811, 30420,   209,  4118,   187, 40292, 44795,
          6088, 15244, 20855, 22701,   243,  6449, 34179, 20557,   215,  5444,
         41140,   765,   100, 36313,   163,   125,   121, 14945,   113, 18309,
         14016, 13973, 13129,  5151,  4340,   187,   187,  4118,   209, 40292,
         44795,  6088, 15244, 20855, 22701,   243,   209,  4118,   187,    14,
          5729,    14, 31158,  3082,   326,  3037,   432,  9305,  2505,   452,
         10532,  1025,   427, 13010,   285,  4384,    14,  7527, 18406,   273,
          2505,  2085,   625, 19737, 20446,   685,  1029,    14, 15177,  9539,
            14, 22027,   427, 13010, 15302,    15,   209,   187,    14, 14673,
          3448, 20446,   476,   320,   908,   275,  4382,  8113,   323,  6891,
           285,   973,    14, 44490,  8892,    13,   824,   347, 21565, 10882,
          8695,    14,  4919, 41035,  3544,   327, 19614,  3888,   285,   638,
            14, 31158,  3210,   281,  3283,   253,  

In [64]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

output = model.generate(
    input_ids=tokens,
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.1,
)

KeyboardInterrupt: 

In [60]:
translated_text = tokenizer.decode(output[0])
print(translated_text)


### 指示 ###
対象とする文章を制約に基づいて翻訳してください。

### 文章の制約 ###
- 日本語で出力してください。
- 箇条書きの文章ごとに翻訳してください。
- 1つの項目ごとに100文字以内で翻訳してください。
- 箇条書きで出力してください。

### 対象とする文章 ###
- Pre-training methods that learn from raw text have revolutionized NLP and web-scale collections of text provide more aggregate supervision than high-quality crowd-labeled NLP datasets. 
- Natural language supervision can be used in computer vision for narrow and well-targeted tasks, such as predicting ImageNet-related hashtags on Instagram images and pre-training models to predict the classes of the noisily labeled JFT-300M dataset. 
- CLIP, a method of learning from natural language supervision, is an efficient way to perform a wide set of tasks during pre-training, including OCR, geo-localization, and action recognition. CLIP is more efficient at zero-shot transfer than an image caption baseline and is more robust than equivalent accuracy supervised ImageNet models.

### 出力 ###
- 日本語で出力してください。
- 箇条書きの文章ごとに翻訳してください。
- 1つの項目ごとに100文字以内